In [128]:
import pandas as pd
import os
import folium

In [129]:
path = os.path.join(".", "people_1908.csv")
people_1908 = pd.read_csv(path)

In [130]:
people_1908 = people_1908.fillna("")

In [131]:
paris_coord = [48.864716, 2.349014]
paris_1908 = folium.Map(location=paris_coord)
paris_1908

In [176]:
#pip install geopy
from geopy.geocoders import Nominatim
from folium.plugins import MarkerCluster

In [138]:
people = people_1908[:30].reset_index()[['Last name', 'First name', 'address']]
people

,Last name,First name,address
0,ABRADIE-d’ARRAST,Michel-Robert,5S bis rue Jouffroy
1,ABBATUCCI,,103 rue La Boétie
2,ABBATUCCI,,32 rue Washington
3,ABOILARI,Georges,46 avenue de Breteuil
4,ABOVILLE et Mme née IIENNE-CART,Christian,23 rue de Grenelle
5,RA et Mme née KNITTEL,Alt Henri,28 rue Copernic
6,ABRANTÈS,,12 ave- nue Henri-Martin
7,ABRANTÈS,Duc,12 avenue Henri- Martin
8,ABRANTÈS,,12 avenue Henri-Martin
9,ABRANTÈS,Cte,12 avenue Henri-Martin


In [148]:
people["latitude"] = "0"
people["longitude"] = "0"
geolocator = Nominatim(user_agent="agent")
for i in range(len(people)):
    #time.sleep(0.1)
    try:
        coord = geolocator.geocode(people['address'][i])
    except: 
        print("Try again...")
        try:
            coord = geolocator.geocode(people['address'][i])
        except:
            print("I give up")
    if(coord):
        people["latitude"][i] = coord.latitude
        people["longitude"][i] = coord.longitude

people

Try again...
Try again...
Try again...
Try again...
Try again...


,Last name,First name,address,latitude,longitude
0,ABRADIE-d’ARRAST,Michel-Robert,5S bis rue Jouffroy,0,0
1,ABBATUCCI,,103 rue La Boétie,48.8713,2.30717
2,ABBATUCCI,,32 rue Washington,48.8736,2.30377
3,ABOILARI,Georges,46 avenue de Breteuil,48.8501,2.31147
4,ABOVILLE et Mme née IIENNE-CART,Christian,23 rue de Grenelle,48.8534,2.32823
5,RA et Mme née KNITTEL,Alt Henri,28 rue Copernic,50.9971,2.38066
6,ABRANTÈS,,12 ave- nue Henri-Martin,0,0
7,ABRANTÈS,Duc,12 avenue Henri- Martin,43.5999,1.3519
8,ABRANTÈS,,12 avenue Henri-Martin,43.5999,1.3519
9,ABRANTÈS,Cte,12 avenue Henri-Martin,43.5999,1.3519


In [177]:
mc = MarkerCluster()


for i in range(len(people)):
    name = people["Last name"][i] + " " + people["First name"][i]
    address = people["address"][i]
    
    html= """
        <h1> """+ name + """</h1>
        <img src = "https://workingwithdog.com/wp-content/uploads/2016/05/new_instagram_logo-1024x1024.jpg" width = 50 height = 50/>
        <p> Influencer of the past </p>
        <h2> """ + address + """ </h2"""
    
    iframe = folium.IFrame(html=html, width=400, height=200)
    popup = folium.Popup(iframe, max_width=2650)
    mc.add_child(folium.Marker([people["latitude"][i], people["longitude"][i]], popup=popup))
    
paris_1908.add_child(mc)

In [178]:
paris_1908

In [114]:
geolocator = Nominatim(user_agent="Test")
geolocator.geocode("74 rue Michel-Ange [Ht 239.81]—et fljj le Prieuré Saint-Michel à Saint- Pair-sur-la-Mer ")